In [1]:
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta
import requests
import pickle
import os
import sys
sys.path.append(os.path.abspath("..")) 

from helpers import tankathon_bballgm_team_mapper
from data_retrieval.realgm_retr import get_realgm_allstar_rosters, get_realgm_allstar_roster

# Lottery Luck Adjustment

In [45]:
def get_draft_lottery_history(year_start, year_end, store_data: bool =True) -> dict[str, pd.DataFrame]:
    draft_lottery_results = {}
    years = list(range(year_start, year_end + 1))
    for year in years:

        # Extract the table for the current year
        url_format = "https://basketball.realgm.com/nba/draft/lottery_results/{year}"
        url = url_format.format(year=year)
        res = requests.get(url)
        soup = BeautifulSoup(res.text, 'html.parser')
        table = soup.find("table", class_="table-striped")

        if not table:
            print(f"No table found for year {year}")
            continue

        # convert the table to a DataFrame
        headers = [th.text.strip() for th in table.find_all("th")]
        rows = []
        for tr in table.find_all("tr")[1:]:  # Skip header
            row = [td.text.strip() for td in tr.find_all("td")]
            if row:
                rows.append(row)
        df = pd.DataFrame(rows, columns=headers)

        # Convert the "Pick Change" column to integers
        df["Pick Change"] = df["Pick Change"].str.replace("+", "", regex=False).astype(int)

        draft_lottery_results[year] = df

    if store_data:
        for year, df in draft_lottery_results.items():
            df.to_csv(f"data/draft_lottery_results_{year}.csv", index=False)
        # Save the results to a pickle file
        with open(f"data/draft_lottery_results_{year_start}_{year_end}.pkl", "wb") as f:
            pickle.dump(draft_lottery_results, f)

    return draft_lottery_results

In [46]:
# collect draft lottery history data
max_year = datetime.now().year
min_year = max_year - 3
draft_lottery_history = get_draft_lottery_history(min_year, max_year - 1, store_data=True)

In [30]:
def get_current_lottery_odds() -> pd.DataFrame:

    url = 'https://www.tankathon.com/'
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    table = soup.find("table", class_="draft-board")

    # convert the table to a DataFrame
    header_row = soup.find("tr", class_="headers")
    headers = [
        td.text.strip()
        for td in header_row.find_all("td")
        if 'mobile' not in td.get('class', [])
        and td.text.strip()  # Ignore empty cells
    ]
    rows = []
    for tr in table.find_all("tr", class_="pick-row-lottery"):
        tds = tr.find_all("td")
        row = []

        for i, td in enumerate(tds):
            # Special handling for team name column
            if "name" in td.get("class", []):
                desktop_team = td.find("div", class_="desktop")
                team_name = desktop_team.text.strip() if desktop_team else ""
                row.append(team_name)
            else:
                row.append(td.text.strip())

        if row:
            rows.append(row)
    return pd.DataFrame(rows, columns=headers)

In [ ]:
# Load the current lottery odds
df_current = get_current_lottery_odds()
df_current['Team'] = df_current.Team.apply(lambda x: tankathon_bballgm_team_mapper[x])
df_current

,Pick,Team,Record,Win%,GB,Streak,L10,Top 4,#1 Ovr
0,1,Utah Jazz,16-59,.213,--,Lost 5,1-9,52.1%,14.0%
1,2,Washington Wizards,16-57,.219,1.0,Lost 1,3-7,52.1%,14.0%
2,3,Charlotte Hornets,18-55,.247,3.0,Lost 4,3-7,52.1%,14.0%
3,4,New Orleans Pelicans,20-54,.270,4.5,Lost 1,3-7,48.1%,12.5%
4,5,Brooklyn Nets,23-51,.311,7.5,Lost 6,1-9,42.1%,10.5%
5,6,Philadelphia 76ers,23-50,.315,8.0,Lost 6,1-9,37.2%,9.0%
6,7,Toronto Raptors,27-47,.365,11.5,Won 3,6-4,32.0%,7.5%
7,8,San Antonio Spurs,31-41,.431,16.5,Lost 2,5-5,26.3%,6.0%
8,9,Portland Trail Blazers,32-42,.432,16.5,Lost 3,4-6,20.3%,4.5%
9,10,Miami Heat,32-41,.438,17.0,Won 3,3-7,13.9%,3.0%


In [66]:
# join lottery team previous lottery results
df_joined = df_current.copy()
for year in draft_lottery_history:
    df_joined = df_joined.join(
        draft_lottery_history[year].set_index('Team')["Pick Change"], on='Team', rsuffix=f'_{year}'
    )
df_joined

,Pick,Team,Record,Win%,GB,Streak,L10,Top 4,#1 Ovr,Pick Change,Pick Change_2023,Pick Change_2024
0,1,Utah Jazz,16-59,.213,--,Lost 5,1-9,52.1%,14.0%,NaN,0.0,-2.0
1,2,Washington Wizards,16-57,.219,1.0,Lost 1,3-7,52.1%,14.0%,0.0,0.0,0.0
2,3,Charlotte Hornets,18-55,.247,3.0,Lost 4,3-7,52.1%,14.0%,0.0,2.0,-3.0
3,4,New Orleans Pelicans,20-54,.270,4.5,Lost 1,3-7,48.1%,12.5%,NaN,0.0,NaN
4,5,Brooklyn Nets,23-51,.311,7.5,Lost 6,1-9,42.1%,10.5%,NaN,NaN,6.0
5,6,Philadelphia 76ers,23-50,.315,8.0,Lost 6,1-9,37.2%,9.0%,NaN,NaN,NaN
6,7,Toronto Raptors,27-47,.365,11.5,Won 3,6-4,32.0%,7.5%,NaN,0.0,-2.0
7,8,San Antonio Spurs,31-41,.431,16.5,Lost 2,5-5,26.3%,6.0%,0.0,2.0,1.0
8,9,Portland Trail Blazers,32-42,.432,16.5,Lost 3,4-6,20.3%,4.5%,-1.0,2.0,-3.0
9,10,Miami Heat,32-41,.438,17.0,Won 3,3-7,13.9%,3.0%,NaN,NaN,NaN


In [67]:
def convert_odds_to_balls(df: pd.DataFrame) -> pd.DataFrame:
    # Convert the odds to integers
    df['start_balls'] = df['#1 Ovr'].replace({'%': ''}, regex=True).astype(float) * 10
    return df
df_joined = convert_odds_to_balls(df_joined)
df_joined

,Pick,Team,Record,Win%,GB,Streak,L10,Top 4,#1 Ovr,Pick Change,Pick Change_2023,Pick Change_2024,start_balls
0,1,Utah Jazz,16-59,.213,--,Lost 5,1-9,52.1%,14.0%,NaN,0.0,-2.0,140.0
1,2,Washington Wizards,16-57,.219,1.0,Lost 1,3-7,52.1%,14.0%,0.0,0.0,0.0,140.0
2,3,Charlotte Hornets,18-55,.247,3.0,Lost 4,3-7,52.1%,14.0%,0.0,2.0,-3.0,140.0
3,4,New Orleans Pelicans,20-54,.270,4.5,Lost 1,3-7,48.1%,12.5%,NaN,0.0,NaN,125.0
4,5,Brooklyn Nets,23-51,.311,7.5,Lost 6,1-9,42.1%,10.5%,NaN,NaN,6.0,105.0
5,6,Philadelphia 76ers,23-50,.315,8.0,Lost 6,1-9,37.2%,9.0%,NaN,NaN,NaN,90.0
6,7,Toronto Raptors,27-47,.365,11.5,Won 3,6-4,32.0%,7.5%,NaN,0.0,-2.0,75.0
7,8,San Antonio Spurs,31-41,.431,16.5,Lost 2,5-5,26.3%,6.0%,0.0,2.0,1.0,60.0
8,9,Portland Trail Blazers,32-42,.432,16.5,Lost 3,4-6,20.3%,4.5%,-1.0,2.0,-3.0,45.0
9,10,Miami Heat,32-41,.438,17.0,Won 3,3-7,13.9%,3.0%,NaN,NaN,NaN,30.0


In [71]:
# lottery luck adjustment
def adjust_lottery_luck(df: pd.DataFrame) -> pd.DataFrame:
    # Adjust the lottery luck based on the previous year's pick change
    df['balls_lottery_luck_adjusted'] = df['start_balls'].copy()
    pick_change_cols = [col for col in df.columns if 'Pick Change_' in col]
    for pick_change_col in pick_change_cols:
        year = pick_change_col.split('_')[-1]
        normalizer = datetime.now().year - int(year)
        df['balls_lottery_luck_adjusted'] -= (df[pick_change_col].fillna(0).astype(int)*10) / normalizer
    return df
df_adjusted = adjust_lottery_luck(df_joined.copy())
df_adjusted['#1 Ovr Adjusted'] = (df_adjusted['balls_lottery_luck_adjusted'] / df_adjusted['balls_lottery_luck_adjusted'].sum() * 100).round(2).astype(str) + '%'
df_adjusted.sort_values(by='balls_lottery_luck_adjusted', ascending=False, inplace=True)
df_adjusted

,Pick,Team,Record,Win%,GB,Streak,L10,Top 4,#1 Ovr,Pick Change,Pick Change_2023,Pick Change_2024,start_balls,balls_lottery_luck_adjusted,#1 Ovr Adjusted
0,1,Utah Jazz,16-59,.213,--,Lost 5,1-9,52.1%,14.0%,NaN,0.0,-2.0,140.0,160.0,16.0%
2,3,Charlotte Hornets,18-55,.247,3.0,Lost 4,3-7,52.1%,14.0%,0.0,2.0,-3.0,140.0,160.0,16.0%
1,2,Washington Wizards,16-57,.219,1.0,Lost 1,3-7,52.1%,14.0%,0.0,0.0,0.0,140.0,140.0,14.0%
3,4,New Orleans Pelicans,20-54,.270,4.5,Lost 1,3-7,48.1%,12.5%,NaN,0.0,NaN,125.0,125.0,12.5%
6,7,Toronto Raptors,27-47,.365,11.5,Won 3,6-4,32.0%,7.5%,NaN,0.0,-2.0,75.0,95.0,9.5%
5,6,Philadelphia 76ers,23-50,.315,8.0,Lost 6,1-9,37.2%,9.0%,NaN,NaN,NaN,90.0,90.0,9.0%
8,9,Portland Trail Blazers,32-42,.432,16.5,Lost 3,4-6,20.3%,4.5%,-1.0,2.0,-3.0,45.0,65.0,6.5%
4,5,Brooklyn Nets,23-51,.311,7.5,Lost 6,1-9,42.1%,10.5%,NaN,NaN,6.0,105.0,45.0,4.5%
7,8,San Antonio Spurs,31-41,.431,16.5,Lost 2,5-5,26.3%,6.0%,0.0,2.0,1.0,60.0,40.0,4.0%
9,10,Miami Heat,32-41,.438,17.0,Won 3,3-7,13.9%,3.0%,NaN,NaN,NaN,30.0,30.0,3.0%


In [73]:
test = df_adjusted[['Team', '#1 Ovr', '#1 Ovr Adjusted']]

# All Star Rest Penalty Adjustment

In [78]:
url = 'https://www.basketball-reference.com/allstar/NBA_2025.html'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')
tables = soup.find_all("table", class_="stats_table")

# convert the table to a DataFrame
# header_row = soup.find("tr", class_="headers")
# headers = [
#     td.text.strip()
#     for td in header_row.find_all("td")
#     if 'mobile' not in td.get('class', [])
#     and td.text.strip()  # Ignore empty cells
# ]
# rows = []
# for tr in table.find_all("tr", class_="pick-row-lottery"):
#     tds = tr.find_all("td")
#     row = []

#     for i, td in enumerate(tds):
#         # Special handling for team name column
#         if "name" in td.get("class", []):
#             desktop_team = td.find("div", class_="desktop")
#             team_name = desktop_team.text.strip() if desktop_team else ""
#             row.append(team_name)
#         else:
#             row.append(td.text.strip())

#     if row:
#         rows.append(row)

In [6]:
max_year = datetime.now().year
min_year = max_year - 3
years = list(range(min_year, max_year + 1))
df = get_realgm_allstar_rosters(years)
# df = get_realgm_allstar_roster(years[0])

In [7]:
df

,Player,Pos,HT,WT,Team,Selection Type,NBA Draft Status,Nationality
0,Jarrett Allen,C,6-9,243,Cleveland Cavaliers,Eastern All-Star Replacement Selection,2017 Rnd 1 Pick 22,United States
1,Giannis Antetokounmpo,F,6-11,243,Milwaukee Bucks,Eastern All-Star Fan Vote Selection,2013 Rnd 1 Pick 15,Greece Nigeria
2,Jimmy Butler III,GF,6-7,230,Miami Heat,Eastern All-Star Coaches Selection,2011 Rnd 1 Pick 30,United States
3,Stephen Curry,G,6-2,185,Golden State Warriors,Western All-Star Fan Vote Selection,2009 Rnd 1 Pick 7,United States
4,DeMar DeRozan,GF,6-6,220,Chicago Bulls,Eastern All-Star Fan Vote Selection,2009 Rnd 1 Pick 9,United States
...,...,...,...,...,...,...,...,...
190,Jayson Tatum,SF,6-8,210,Boston Celtics,Eastern All-Star Fan Vote Selection,2017 Rnd 1 Pick 3,United States
191,Karl-Anthony Towns,C,7-0,248,New York Knicks,Eastern All-Star Fan Vote Selection,2015 Rnd 1 Pick 1,United States Dominican Republic
192,Victor Wembanyama,F,7-3,235,San Antonio Spurs,Western All-Star Coaches Selection,2023 Rnd 1 Pick 1,France
193,Jalen Williams,G,6-5,211,Oklahoma City Thunder,Western All-Star Coaches Selection,2022 Rnd 1 Pick 12,United States


In [90]:
from io import StringIO


base_url = "https://basketball.realgm.com/nba/allstar/game/rosters"

# years = list(range(1951, 2025))
year=2024
# Send a GET request to the page URL
response = requests.get(f"{base_url}/{year}")

if response.status_code == 200:
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, "html.parser")
    tables = soup.find_all("table", class_="table-compact")

    year_df_list = []
    for table in tables:
        year_df_list.append(pd.read_html(StringIO(str(table)))[0])


In [92]:
year_df_list[0]

,Player,Pos,HT,WT,Team,Selection Type,NBA Draft Status,Nationality
0,Bam Adebayo,C,6-9,255,Miami Heat,Eastern All-Star Coaches Selection,2017 Rnd 1 Pick 14,United States
1,Giannis Antetokounmpo,F,6-11,243,Milwaukee Bucks,Eastern All-Star Fan Vote Selection,2013 Rnd 1 Pick 15,Greece Nigeria
2,Paolo Banchero,C,6-10,250,Orlando Magic,Eastern All-Star Coaches Selection,2022 Rnd 1 Pick 1,United States Italy
3,Scottie Barnes,SF,6-7,237,Toronto Raptors,Eastern All-Star Replacement Selection,2021 Rnd 1 Pick 4,United States
4,Jaylen Brown,SF,6-6,223,Boston Celtics,Eastern All-Star Coaches Selection,2016 Rnd 1 Pick 3,United States
5,Jalen Brunson,PG,6-2,190,New York Knicks,Eastern All-Star Coaches Selection,2018 Rnd 2 Pick 3,United States
6,Joel Embiid,C,7-0,280,Philadelphia Sixers,Eastern All-Star Fan Vote Selection,2014 Rnd 1 Pick 3,Cameroon United States
7,Tyrese Haliburton,PG,6-5,185,Indiana Pacers,Eastern All-Star Fan Vote Selection,2020 Rnd 1 Pick 12,United States
8,Damian Lillard,G,6-2,195,Milwaukee Bucks,Eastern All-Star Fan Vote Selection,2012 Rnd 1 Pick 6,United States
9,Tyrese Maxey,PG,6-2,200,Philadelphia Sixers,Eastern All-Star Coaches Selection,2020 Rnd 1 Pick 21,United States
